# Preprocess

Programa compilado en Collab, que contiene el preprocesado de datos. 
- Lee datos.
- Realiza las transformaciones.
- Descarga un csv mensual por mes en local.

### Leer datos

In [ ]:
import os
import pandas as pd
import datetime
import glob

i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))

for year in [2022]:
    for month, month_name in i2m:
        os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
        os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

fileList = os.listdir('Path_Data')
listCsv = [file for file in fileList if file[-3:] == "csv"]


### Procesar y transformar archivos CSV

In [ ]:
for i in range(0, 12, 1):
    ftemp = pd.read_csv(listCsv[i])
    
    ftemp['last_reported'] = pd.to_datetime(ftemp['last_reported'], unit='s')
    ftemp['last_updated'] = pd.to_datetime(ftemp['last_updated'], unit='s')
    
    ftemp['year'] = ftemp.last_reported.dt.year
    ftemp['month'] = ftemp.last_reported.dt.month
    ftemp = ftemp[ftemp['month'] == i + 3]
    ftemp['hour'] = ftemp.last_reported.dt.hour
    ftemp['dayyear'] = ftemp.last_reported.dt.day_of_year
    ftemp['houryear'] = (ftemp.dayyear - 1) * 24 + ftemp.hour
    
    # Calcular la suma de las columnas "num_docks_available" y "num_bikes_available" en una nueva columna "Llocs", son la suma de bicis y sitios bacios
    ftemp['Llocs'] = ftemp['num_docks_available'] + ftemp['num_bikes_available']
    
    # Configurar columna "last_reported" como índice, es la fecha de la fila
    ftemp = ftemp.set_index('last_reported')
    
    # Agrupar por "station_id" y "houryear" y calcular la media de los valores para cada grupo
    ftemp = ftemp.groupby(['station_id', 'houryear']).mean().reset_index()
    ftemp = ftemp.reset_index()
    
    
    ftemp['houryear'] = ftemp['houryear'].astype(int)
    ftemp['year'] = ftemp['year'].astype(int)
    
    # Fecha y hora como una cadena con formato 'AAAA/MM/DD HH:MM:SS'
    ftemp['time'] = pd.to_datetime(ftemp['year'].astype(str), format='%Y') + pd.to_timedelta(ftemp['houryear'], unit='h')
    ftemp['time'] = ftemp['time'].dt.strftime('%Y/%m/%d %H:%M:%S')
    ftemp['dayofweek'] = ftemp['time'].dt.dayofweek
    ftemp['time2'] = pd.to_datetime(ftemp['time'].dt.date)
    
    ftemp.to_csv('2019' + str(i + 1) + '.csv', index=False)
    files.download('/content/2019' + str(i + 1) + '.csv')
    
    del ftemp

### Añadir información Precipitación y Viento

In [ ]:
fileList = os.listdir('Path_Data')
listCsv = [file for file in fileList if file[-3:] == "csv"]


for i in range(0, len(listCsv), 1):
    
    ftemp = pd.read_csv(listCsv[i])
    df = pd. read_csv ('Precipitacio.csv')
    df=df.drop(['CODI_ESTACIO','CODI_BASE'],axis=1)
    df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
    df['year'] = df.time.dt.year
    df['dayyear'] = df.time.dt.day_of_year
    
    #Creamos una variable que indique si ha llovido ese dia ponga un 1, indicando lluvia en ese dia
    df=df.groupby(['year','dayyear']).sum()
    df['Rain']=np.where(df['VALOR_LECTURA']>1,1,0)
    
    #Rain_Lectura será el valor real de la lectura de lluvia en esa hora
    df['Rain_Lectura']=df['VALOR_LECTURA']
    df=df.drop(['VALOR_LECTURA'],axis=1)
    df=df.reset_index()
    
    df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
    
    #Unimos la informacion del csv original con el de precipitacion
    Total=pd.merge_ordered(ftemp,df,how='left',on=['time2'],fill_method='ffill')
    Total=Total.drop(['year_y','dayyear_y'],axis=1)
    
    #Realizamos lo mismo con los valores de viento, en ese caso si el viento es superior a 20 m/s en algun momento
    #se rellena la variable WIND con un 1 todo el dia para indicar que era un día con viento.
    
    df = pd. read_csv ('Vent.csv')
    df=df.drop(['CODI_ESTACIO','CODI_BASE'],axis=1)
    df['time'] = pd.to_datetime(df['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
    df['year'] = df.time.dt.year
    df['dayyear'] = df.time.dt.day_of_year
    df=df.groupby(['year','dayyear']).max()
    df['Wind']=np.where(df['VALOR_LECTURA']>20,1,0)
    df['Wind_Lectura']=df['VALOR_LECTURA']
    df=df.drop(['VALOR_LECTURA'],axis=1)
    df=df.drop(['DATA_LECTURA','time'],axis=1)
    df=df.reset_index()
    df['time2'] = pd.to_datetime(df['year'].astype(str) + df['dayyear'].astype(str), format='%Y%j')
    Total=pd.merge_ordered(Total,df,how='left',on=['time2'],fill_method='ffill')
    
    #guardamos los documentos
    Total.to_csv(listCsv+'_PV.csv', index=False)